In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

The syntax of the command is incorrect.
'cp' is not recognized as an internal or external command,
operable program or batch file.
'chmod' is not recognized as an internal or external command,
operable program or batch file.


In [3]:
import pandas as pd

In [4]:
from zipfile import ZipFile
dataset="F:\ML Projects\Twitter_sentiment_analysis\Archive.zip"
with ZipFile(dataset,'r')as zip:
    zip.extractall()
    print('The dataset is extracted')

<>:2: SyntaxWarning: invalid escape sequence '\M'
<>:2: SyntaxWarning: invalid escape sequence '\M'
C:\Users\Admin\AppData\Local\Temp\ipykernel_18940\2963427494.py:2: SyntaxWarning: invalid escape sequence '\M'
  dataset="F:\ML Projects\Twitter_sentiment_analysis\Archive.zip"


The dataset is extracted


In [1]:
import numpy as np
import pandas as pd
import re #regular expression lib
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [6]:
twitter_data = pd.read_csv("F:\\ML Projects\\Twitter_sentiment_analysis\\Data.csv", encoding='ISO-8859-1')

In [7]:
twitter_data.shape

(1599999, 6)

In [8]:
twitter_data.sample(5)

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
208567,0,1973756939,Sat May 30 11:46:27 PDT 2009,NO_QUERY,MMofOz,@lownleeeynjul That sucks lisa cuz u studied a...
1568808,4,2188284207,Mon Jun 15 21:37:10 PDT 2009,NO_QUERY,tallivansunder,@QueenofHaiku I'm enjoying it / I am still on ...
1047715,4,1958074277,Fri May 29 02:34:20 PDT 2009,NO_QUERY,longestpull,The Longest Pull Website will be closing on 30...
660770,0,2242831368,Fri Jun 19 12:41:11 PDT 2009,NO_QUERY,MariaWiddows,There is not enough wine in my house this even...
1080633,4,1968317155,Fri May 29 21:11:26 PDT 2009,NO_QUERY,Flavorofdasouth,@_kimcaldwell You're doing what you love! Not...


In [9]:
#twitter_data.columns= ['target','id','date','flag','user','text']

column_names = ["target", "id", "date", "flag", "user", "text"]  # Adjust names if needed


twitter_data = pd.read_csv(
    "F:\\ML Projects\\Twitter_sentiment_analysis\\Data.csv",
    encoding='ISO-8859-1',
    names=column_names,  
    header=None  
)


twitter_data.head() 


,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [10]:
twitter_data.head()

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [11]:
twitter_data.shape

(1600000, 6)

In [ ]:
twitter_data.isnull().sum()

target    0
id        0
date      0
flag      0
user      0
text      0
dtype: int64

In [13]:
twitter_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600000 entries, 0 to 1599999
Data columns (total 6 columns):
 #   Column  Non-Null Count    Dtype 
---  ------  --------------    ----- 
 0   target  1600000 non-null  int64 
 1   id      1600000 non-null  int64 
 2   date    1600000 non-null  object
 3   flag    1600000 non-null  object
 4   user    1600000 non-null  object
 5   text    1600000 non-null  object
dtypes: int64(2), object(4)
memory usage: 73.2+ MB


In [14]:
twitter_data['target'].value_counts()

target
0    800000
4    800000
Name: count, dtype: int64

In [15]:
twitter_data.replace({'target':{4:1}},inplace=True)

In [16]:
twitter_data['target'].value_counts()

target
0    800000
1    800000
Name: count, dtype: int64

Stemming

In [17]:
port_stem = PorterStemmer()

In [18]:
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z\s]',' ',content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

<>:2: SyntaxWarning: invalid escape sequence '\s'
<>:2: SyntaxWarning: invalid escape sequence '\s'
C:\Users\Admin\AppData\Local\Temp\ipykernel_18940\1602630669.py:2: SyntaxWarning: invalid escape sequence '\s'
  stemmed_content = re.sub('[^a-zA-Z\s]',' ',content)


In [19]:
import nltk
nltk.download('stopwords')


[nltk_data] Error loading stopwords: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


False

In [20]:
print(stopwords.words('english'))

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

In [ ]:
'''
import swifter

twitter_data['stemmed_content'] = twitter_data['text'].swifter.apply(stemming)
'''

Pandas Apply:   0%|          | 0/1600000 [00:00<?, ?it/s]

In [22]:
twitter_data['text'][0]

"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D"

In [25]:
twitter_data['stemmed_content'][0]

'switchfoot http twitpic com zl awww bummer shoulda got david carr third day'

In [ ]:
#twitter_data.to_csv("F:\\ML Projects\\Twitter_sentiment_analysis\\Processed_Data.csv", index=False)


In [2]:
twitter_data = pd.read_csv("F:\\ML Projects\\Twitter_sentiment_analysis\\Processed_Data.csv")

In [3]:
twitter_data.head()

,target,id,date,flag,user,text,stemmed_content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot http twitpic com zl awww bummer sho...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset updat facebook text might cri result sch...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichan dive mani time ball manag save rest g...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclass behav mad see


In [4]:
X= twitter_data['stemmed_content'].values
Y= twitter_data['target'].values

In [5]:
X

array(['switchfoot http twitpic com zl awww bummer shoulda got david carr third day',
       'upset updat facebook text might cri result school today also blah',
       'kenichan dive mani time ball manag save rest go bound', ...,
       'readi mojo makeov ask detail',
       'happi th birthday boo alll time tupac amaru shakur',
       'happi charitytuesday thenspcc sparkschar speakinguph h'],
      dtype=object)

In [6]:
Y

array([0, 0, 0, ..., 1, 1, 1])

In [7]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.2, stratify=Y, random_state=2)

In [8]:
X.shape,X_train.shape,X_test.shape

((1600000,), (1280000,), (320000,))

In [9]:
Y.shape,Y_train.shape,Y_test.shape

((1600000,), (1280000,), (320000,))

textual-numerical via vectorizer(tfidf)

In [ ]:

X_train = pd.Series(X_train).fillna("").astype(str).tolist()
X_test = pd.Series(X_test).fillna("").astype(str).tolist()


In [11]:
vectorizer = TfidfVectorizer()
X_train= vectorizer.fit_transform(X_train)
X_test= vectorizer.transform(X_test)


In [12]:
print(X_train)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 9453092 stored elements and shape (1280000, 461488)>
  Coords	Values
  (0, 436713)	0.27259876264838384
  (0, 354543)	0.3588091611460021
  (0, 185193)	0.5277679060576009
  (0, 109306)	0.3753708587402299
  (0, 235045)	0.41996827700291095
  (0, 443066)	0.4484755317023172
  (1, 160636)	1.0
  (2, 109306)	0.4591176413728317
  (2, 124484)	0.1892155960801415
  (2, 407301)	0.18709338684973031
  (2, 129411)	0.29074192727957143
  (2, 406399)	0.32105459490875526
  (2, 433560)	0.3296595898028565
  (2, 77929)	0.31284080750346344
  (2, 443430)	0.3348599670252845
  (2, 266729)	0.24123230668976975
  (2, 409143)	0.15169282335109835
  (2, 178061)	0.1619010109445149
  (2, 150715)	0.18803850583207948
  (2, 132311)	0.2028971570399794
  (2, 288470)	0.16786949597862733
  (3, 406399)	0.29029991238662284
  (3, 158711)	0.4456939372299574
  (3, 151770)	0.278559647704793
  (3, 56476)	0.5200465453608686
  :	:
  (1279996, 318303)	0.21254698865277744
  (12

Training(Logistic Regression)

In [13]:
model = LogisticRegression(max_iter=1000)

In [14]:
model.fit(X_train,Y_train)

LogisticRegression(max_iter=1000)

In [15]:
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction,Y_train)

In [16]:
print('Accuracy score of training data : ', training_data_accuracy)

Accuracy score of training data :  0.79871953125


In [17]:
X_test_prediction = model.predict(X_test)
training_data_accuracy = accuracy_score(X_test_prediction,Y_test)

In [18]:
print('Accuracy score of testing data : ', training_data_accuracy)

Accuracy score of testing data :  0.77668125


In [19]:
import pickle

In [20]:
filename ='trained_model.sav'
pickle.dump(model,open(filename,'wb'))

In [21]:
loaded_model = pickle.load(open('F:\\ML Projects\\Twitter_sentiment_analysis\\trained_model.sav','rb'))

In [24]:
loaded_model

LogisticRegression(max_iter=1000)

In [40]:
X_new = X_test[100000]
print(Y_test[100000])

prediction = loaded_model.predict(X_new)
print(prediction)

if (prediction[0]==0):
    print('Negative Tweet')
else:
    print('Positive Tweet')

0
[0]
Negative Tweet
